In [ ]:
import pandas as pd
import os
 
# Caminho dos CSVs
caminho_csv = 'C:\\AFL\\csv_parser\\DATA_EXCEL\\sis'  
caminho_parquet = 'C:\\AFL\\csv_parser\\DATA_PARQUET\\sis\\NF-e Detalhamento de Produtos e Serviços e Tributos'  
 
# Criar o caminho de saída, se não existir
if not os.path.exists(caminho_parquet):
    os.makedirs(caminho_parquet)
 
# Lista para armazenar DataFrames
dfs_paginas = {}
 
# Iterar pelos arquivos CSV no diretório
for arquivo in os.listdir(caminho_csv):
    if arquivo.endswith('.csv'):
        # Caminho completo do arquivo CSV
        caminho_csv = os.path.join(caminho_csv, arquivo)
       
        try:
            # Lê o arquivo CSV em um DataFrame
            df = pd.read_csv(caminho_csv, delimiter="|", dtype=str)
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')
            continue
       
        # Extrai o nome base antes da parte de paginação, se houver
        nome_base = arquivo.split('_pagina')[0]
       
        # Se ainda não tiver uma entrada para este nome_base, cria
        if nome_base not in dfs_paginas:
            dfs_paginas[nome_base] = []
       
        # Adiciona o DataFrame à lista de páginas
        if not df.empty:
            dfs_paginas[nome_base].append(df)
            print(f'Lido: {arquivo} com {df.shape[0]} linhas')
 
# Processar e unir os DataFrames por nome_base
for nome_base, lista_dfs in dfs_paginas.items():
    # Se houver mais de uma página, concatena os DataFrames
    if len(lista_dfs) > 1:
        df_final = pd.concat(lista_dfs, ignore_index=True)
        print(f'{nome_base} - Páginas unidas com {df_final.shape[0]} linhas')
    elif lista_dfs:
        df_final = lista_dfs[0]
    else:
        print(f'{nome_base} - Nenhum dado encontrado')
        continue
   
    # Caminho para salvar o arquivo Parquet
    caminho_parquet = os.path.join(caminho_parquet, f'{nome_base}.parquet')
   
    # salva em formato Parquet
    try:
        df_final.to_parquet(caminho_parquet, index=False)
        print(f'Salvo: {caminho_parquet}')
    except Exception as e:
        print(f'Erro ao salvar {nome_base}.parquet: {e}')
 
print('Conversão concluída!')
 

Lido: NF-e Detalhamento de Produtos e Serviços e Tributos.csv com 18552 linhas
Salvo: C:\AFL\csv_parser\DATA_PARQUET\sis\NF-e Detalhamento de Produtos e Serviços e Tributos\NF-e Detalhamento de Produtos e Serviços e Tributos.csv.parquet
Conversão concluída!


In [16]:
import pandas as pd
import pyperclip

# Caminho do arquivo Parquet
caminho_parquet = 'C:\\AFL\\csv_parser\\DATA_PARQUET\\sis\\NF-e Detalhamento de Produtos e Serviços e Tributos'

# Lista de palavras-chave para conversão e substituição
palavras_chave = ["Vlr", "Valor", "vlr", "valor"]  # Modifique conforme necessário

# Ler o arquivo Parquet
df = pd.read_parquet(caminho_parquet)

# Informações do DataFrame
num_linhas, num_colunas = df.shape  # Obter número de linhas e colunas

# Criar a string com os resultados
resultado = f"🔹 Total de Linhas: {num_linhas}\n🔹 Total de Colunas: {num_colunas}\n\n📊 Detalhes das Colunas:\n"

# Percorre todas as colunas e adiciona detalhes
for coluna in df.columns:
    linhas_preenchidas = df[coluna].count()  # Conta valores não nulos
    
    # Verifica se a coluna está na lista de palavras-chave
    if any(palavra.lower() in coluna.lower() for palavra in palavras_chave):
        # Converte para float, ignorando erros caso tenha valores inválidos
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
        
        # Soma os valores da coluna
        soma_valores = df[coluna].sum(skipna=True)
        
        # Substitui "." por "," para manter o padrão brasileiro na exibição
        df[coluna] = df[coluna].astype(str).str.replace(".", ",")
        
        resultado += f"\n📌 Coluna: {coluna} (🔄 Convertida para Float e Substituição de '.' por ',')\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"
        resultado += f"   🔹 Total Somado: {soma_valores:,.2f}\n"
    else:
        resultado += f"\n📌 Coluna: {coluna}\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"

# Copiar para a área de transferência
pyperclip.copy(resultado)

# Exibir os resultados
print(resultado)
print("\n✅ Resultado copiado para a área de transferência!")


🔹 Total de Linhas: 18552
🔹 Total de Colunas: 301

📊 Detalhes das Colunas:

📌 Coluna: Chave NF-e
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Situação
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Data Situação
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Natureza Operação
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Indicador Forma Pagamento
   🔹 Linhas Preenchidas: 11901

📌 Coluna: Modelo
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Série
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Número Documento
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Data Emissão Documento
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Data Entrada/Saída
   🔹 Linhas Preenchidas: 5730

📌 Coluna: Tipo Operação
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Finalidade Emissão
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Indicador Operação Consumidor
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Indicador Presença Comprador
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Tipo Emissão
   🔹 Linhas Preenchidas: 18552

📌 Coluna: Data/Hora Contingência
   🔹 Lin